## Introduction - Business Problem

### Where in London would it be best to open an Italian restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?

## Data 
### Based on definition of our problem, factors that will influence our decision are:

### * number of existing restaurants in the neighborhood (any type of restaurant)
### * number of and distance to Italian restaurants in the neighborhood, if any
### * distance of neighborhood from city center

### We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

### Following data sources will be needed to extract/generate the required information:

### Centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
### Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API
### Coordinate of London center will be obtained using Google Maps API geocoding 

### The data that will be used is Foursquare data for London city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [1]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 962kB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 5.5MB/s eta 0:00:01    |████████████▍                   | 5.7MB 7.3MB/s eta 0:00:02    |██████████████████              | 8.2MB 7.3MB/s eta 0:00:01�████████▌            | 9.0MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 18.6MB/s eta 0:00:01    |███████████████████████▉        | 8.2MB 18.6MB/s eta 0:00:01     |██████████████████████████▏     | 8.9MB 18.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 21.2MB/s eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
     |████████████████████████████████| 112kB 4.8MB/s eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [27]:
address = 'London, UK'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))
London_center = [latitude, longitude]
print (London_center)
print (location)

The geograpical coordinate of London is 51.5073219, -0.1276474.
[51.5073219, -0.1276474]
London, Greater London, England, SW1A 2DX, United Kingdom


In [54]:
CLIENT_ID = '5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1' # your Foursquare ID
CLIENT_SECRET = '5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1
CLIENT_SECRET:5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND


In [6]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1&client_secret=5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND&ll=51.5073219,-0.1276474&v=20180605&query=Restaurant&radius=50000&limit=100'

In [8]:
results = requests.get(url).json()

In [9]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4af41e26f964a52024f021e3,Portrait Restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1593985082,False,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",130427500,NaN
1,5001b8e6e4b05e79b0b3b84d,Bill's Restaurant,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",v-1593985082,False,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",719,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,NaN
2,4afc9a04f964a5204c2422e3,C&R Café Restaurant,"[{'id': '4bf58dd8d48988d156941735', 'name': 'M...",v-1593985082,False,4-5 Rupert Court,NaN,51.511111,-0.132445,"[{'label': 'display', 'lat': 51.51111126738831...",537,W1D 6DY,GB,Chinatown,Greater London,United Kingdom,"[4-5 Rupert Court, Chinatown, Greater London, ...",NaN,Chinatown
3,4b9c54b8f964a520206036e3,Hung's Chinese Restaurant | 美食軒,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1593985082,False,27 Wardour St,NaN,51.511278,-0.131953,"[{'label': 'display', 'lat': 51.51127771538517...",531,W1D 6PR,GB,London,Greater London,United Kingdom,"[27 Wardour St, London, Greater London, W1D 6P...",NaN,NaN
4,4ed23e3b2c5bc35059276f87,London Chinatown Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1593985083,False,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,NaN


In [10]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Portrait Restaurant,Café,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",NaN,4af41e26f964a52024f021e3
1,Bill's Restaurant,English Restaurant,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",719,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,5001b8e6e4b05e79b0b3b84d
2,C&R Café Restaurant,Malay Restaurant,4-5 Rupert Court,NaN,51.511111,-0.132445,"[{'label': 'display', 'lat': 51.51111126738831...",537,W1D 6DY,GB,Chinatown,Greater London,United Kingdom,"[4-5 Rupert Court, Chinatown, Greater London, ...",Chinatown,4afc9a04f964a5204c2422e3
3,Hung's Chinese Restaurant | 美食軒,Chinese Restaurant,27 Wardour St,NaN,51.511278,-0.131953,"[{'label': 'display', 'lat': 51.51127771538517...",531,W1D 6PR,GB,London,Greater London,United Kingdom,"[27 Wardour St, London, Greater London, W1D 6P...",NaN,4b9c54b8f964a520206036e3
4,London Chinatown Restaurant,Chinese Restaurant,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,4ed23e3b2c5bc35059276f87


In [52]:
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [55]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations:

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


 . .

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':


 . .

...

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


 .

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':


 . . done.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [12]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('London center longitude={}, latitude={}'.format(longitude, latitude))
x, y = lonlat_to_xy(longitude, latitude)
print('London center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('London center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
London center longitude=-0.1276474, latitude=51.5073219
London center UTM X=-547012.468459844, Y=5815556.876143048
London center longitude=-0.12764739999999858, latitude=51.5073219


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [13]:
London_center_x, London_center_y = lonlat_to_xy(longitude, latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = London_center_x - 6000
x_step = 600
y_min = London_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(London_center_x, London_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6

364 candidate neighborhood centers generated.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel

In [57]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2930
Total number of Italian restaurants: 419
Percentage of Italian restaurants: 14.30%
Average number of restaurants in neighborhood: 7.321428571428571


In [58]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4f36cda5e4b00af1794f9e73', 'Bistro Union', 51.4543814946221, -0.13988732409507973, '40 Abbeville Rd, London, Greater London, SW4 6NG, United Kingdom', 279, False, -549090.6732396316, 5809897.2585512595)
('532456d1498ea72a7dd6da5d', 'Tokyo Retro', 51.45458799311766, -0.13965004754211566, '57 Abbeville Road, London, Greater London, SW4 9JW, United Kingdom', 263, False, -549069.5063997398, 5809916.570855112)
('4e8617229a52a426022d5fb7', 'The Abbeville Kitchen', 51.45432647310704, -0.1399971114767822, '47 Abbeville Rd., London, Greater London, SW4 0JX, United Kingdom', 287, False, -549099.5213740014, 5809892.791754158)
('58f7ab9fa35dce4c76339235', '33 Abbeville Road', 51.454055, -0.1403, 'London, Greater London, SW4, United Kingdom', 310, False, -549126.7253096951, 5809867.271068442)
('4c0a8d5cffb8c9b613be6b61', 'Newtons Restaurant and Bar', 51.454076, -0.140104, 'Abbeville road, London, Greater London, SW4 9LA, United Kingdom', 296, False,

In [59]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4bf8fe195efe2d7fd6986b34', 'La Baita', 51.458227731276466, -0.14880104944307312, 'Clapham Common North Side, Clapham, Greater London, SW4 0QW, United Kingdom', 131, True, -549614.9818675444, 5810451.410157608)
('59a92e91b2958f5516333c88', 'Canova Hall', 51.46393661129359, -0.11513047452350199, '250 Ferndale Rd, London, Greater London, SW9 8BQ, United Kingdom', 322, True, -547162.3360713229, 5810590.321695896)
('5766787c498e84f6e8288f04', 'Franzina Trattoria', 51.463492, -0.112374, 'United Kingdom', 300, True, -546982.8164496595, 5810501.174047431)
('53369f55498ec0b8cac28896', 'Artusi', 51.46672969666967, -0.07284656623155056, '161 Bellenden Rd, London, Greater London, SE15 4DH, United Kingdom', 146, True, -544184.6752071743, 5810283.805052945)
('4bc9a1e3b6c49c7478b28e91', 'Buonasera', 51.460021247686385, -0.16663788243523334, '22-26 Northcote Rd., London, Greater London, SW11 1NX, United Kingdom', 188, True, -550801.8196322026, 

In [60]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Restaurant Gordon Ramsay, Maze Grill, The Sizzling Squid
Restaurants around location 102: M&G Vip Lounge :-)
Restaurants around location 103: Pimlico Spice
Restaurants around location 104: Khallouk & Taylor, Goya, Pimlico Tandoori, Millbank Spice
Restaurants around location 105: Waterfront London Vauxhall Restaurant, Casa Madeira, Rex Whistler, Pico Bar & Grill, STEAX AND THE CITY
Restaurants around location 106: Ragged Canteen
Restaurants around location 107: Samsun Kebab
Restaurants around location 108: Sabor Peruano, Toulouse Lautrec, Chatkhara, Dragon Castle, Tai Tip Mein, Leños y Carbón, After Taste Chinese Rest. 返寻味, La Bodeguita Restaurant
Restaurants around location 109: Lebanese Grill
Restaurants around location 110: Tower Cafe, Jad Grill, Tower Tandoori, Crystal China 味缘 （伦敦最强麻辣香锅 川菜）, Navarro


In [63]:
map_London = folium.Map(location=London_center, zoom_start=13)
folium.Marker(London_center, popup='Alexanderplatz').add_to(map_London)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_London)
map_London

#### So now we have all the restaurants in area within few kilometers from London, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

#### This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology 
#### In this project we will direct our efforts on detecting areas of London that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

#### In first step we have collected the required data: location and type (category) of every restaurant within 6km from London. We have also identified Italian restaurants (according to Foursquare categorization).

#### Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of London - we will use heatmaps to identify a few promising areas close to center with low number of restaurants in general (and no Italian restaurants in vicinity) and focus our attention on those areas.

#### In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Italian restaurants in radius of 400 meters. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Results and Discussion 
#### Our analysis shows that although there is a great number of restaurants in London (~2000 in our initial area of interest), which offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics and a number of pockets of low restaurant density.


#### Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

#### The results contain the largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to London but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion 
#### Purpose of this project was to identify London areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

#### Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.